<a href="https://colab.research.google.com/github/falon-go-weeee/Colab-Notebooks/blob/main/elasticsearch_api_service_HIGH_ram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download and setup the Elasticsearch instance


In [ ]:
import time
import os

Run the instance as a daemon process

In [ ]:
!mkdir -p /content/ES_data/data

In [ ]:
!sudo chown -R daemon:daemon /content/ES_data/data

In [ ]:
%%bash

wget -c https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.7.0-linux-x86_64.tar.gz
wget -c https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.7.0-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-8.7.0-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-8.7.0/
umount /sys/fs/cgroup
apt install cgroup-tools

In [ ]:
!sed -i 's/#path\.data: \/path\/to\/data/path.data: \/content\/ES_data\/data/g' /content/elasticsearch-8.7.0/config/elasticsearch.yml

In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-8.7.0/bin/elasticsearch

In [ ]:
time.sleep(20)

Once the instance has been started, grep for `elasticsearch` in the processes list to confirm the availability.

In [ ]:
%%bash

ps -ef | grep elasticsearch

In [ ]:
!sed -i 's/xpack.security.enabled: true/xpack.security.enabled: false/g' /content/elasticsearch-8.7.0/config/elasticsearch.yml

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import os
import time

Tunnel_type = "bore" #@param ["ngrok", "bore", "cloudflare"]
port = "34525" #@param {type:"string"}

def install_ngrok():
  if not os.path.exists('/usr/local/lib/python3.8/dist-packages/pyngrok'):
    !pip install pyngrok
    clear_output()

def install_bore():
  if not os.path.exists('/usr/bin/bore'):
    !wget https://github.com/ekzhang/bore/releases/download/v0.4.1/bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !tar -xf bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !rm -f bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !cp bore /usr/bin/bore
    !rm -rf bore
    clear_output()

def install_cloudflare():
  if not os.path.exists('/usr/bin/cloudflared'):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
    clear_output()

if Tunnel_type == "bore":
  install_bore()
  use_ip = ""
  if port:
    # !curl ifconfig.me > ip.txt
    # clear_output()
    # ip = getoutput('cat /content/ip.txt')
    # ip = f"-l {ip}"
    !nohup bore local 9200 --to bore.pub -p $port > /content/tunnel_srv.txt 2>&1 &
  else:
    !nohup bore local 9200 --to bore.pub > /content/tunnel_srv.txt 2>&1 &
  time.sleep(3)
  !grep -o 'bore.pub:[^ ]*' /content/tunnel_srv.txt >/content/tunnel_srvr.txt
  time.sleep(3)
  tunnel_srv= getoutput('cat /content/tunnel_srvr.txt')
  print('http://'+tunnel_srv)
  # !rm /content/tunnel_srv.txt
  !rm /content/tunnel_srvr.txt

elif Tunnel_type == "ngrok":
  install_ngrok()
  from pyngrok import ngrok
  # Terminate ngrok port
  ngrok.kill()
  # Set authentication (optional)
  # Get your authentication token via https://dashboard.ngrok.com/auth
  NGROK_AUTH_TOKEN = "2H79LWzHBonanC1xWxoGXW6fvtY_5rD7uShKgACuSLvUwMvJA"
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)
  public_url = ngrok.connect(9200)
  print(public_url)

elif Tunnel_type == "cloudflare":
  install_cloudflare()
  !pkill cloudflared
  time.sleep(4)
  # Tunnel_link = "& cloudflared tunnel --url http://localhost:7860"
  !nohup cloudflared tunnel --url http://localhost:9200 > /content/srv.txt 2>&1 &
  time.sleep(4)
  !grep -o 'https[^[:space:]]*\.trycloudflare.com' /content/srv.txt >/content/srvr.txt
  srv= getoutput('cat /content/srvr.txt')
  print(srv)
  time.sleep(4)
  # !rm /content/srv.txt /content/srvr.txt

#START elasticsearch service
before running this cell edit config to disable security features

In [ ]:
#@title run ES service
import os
from subprocess import Popen, PIPE, STDOUT


Service_type = "normal" #@param ["normal", "BG service"]

if Service_type == "normal":
    try:
      !sudo -H -u daemon elasticsearch-8.7.0/bin/elasticsearch
    except:
      clear_output()
      !sudo -H -u daemon elasticsearch-8.7.0/bin/elasticsearch
    finally:
      clear_output()
      print("try running this cell again")
elif Service_type == "BG service":
    try:
      # !sudo -H -u daemon elasticsearch-8.7.0/bin/elasticsearch
      es_server = Popen(['elasticsearch-8.7.0/bin/elasticsearch'], 
                      stdout=PIPE, stderr=STDOUT,
                      preexec_fn=lambda: os.setuid(1)  # as daemon
                    )
    except:
      clear_output()
      # !sudo -H -u daemon elasticsearch-8.7.0/bin/elasticsearch
      es_server = Popen(['elasticsearch-8.7.0/bin/elasticsearch'], 
                      stdout=PIPE, stderr=STDOUT,
                      preexec_fn=lambda: os.setuid(1)  # as daemon
                    )
    finally:
      clear_output()

In [ ]:
!ps -ef | grep elasticsearch

##extra config

####postgres

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql.service start

In [ ]:
%env DATABASE_NAME=DB
%env PG_HOST=localhost
%env PG_PORT=5432
%env PG_USER=postgres
%env PG_PASS=postgres

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS tfio_demo;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE grocery;'

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import os
import time

Tunnel_type = "bore" #@param ["ngrok", "bore", "cloudflare"]
port = "34535" #@param {type:"string"}

def install_ngrok():
  if not os.path.exists('/usr/local/lib/python3.8/dist-packages/pyngrok'):
    !pip install pyngrok
    clear_output()

def install_bore():
  if not os.path.exists('/usr/bin/bore'):
    !wget https://github.com/ekzhang/bore/releases/download/v0.4.1/bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !tar -xf bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !rm -f bore-v0.4.1-i686-unknown-linux-musl.tar.gz
    !cp bore /usr/bin/bore
    !rm -rf bore
    clear_output()

def install_cloudflare():
  if not os.path.exists('/usr/bin/cloudflared'):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
    clear_output()

if Tunnel_type == "bore":
  install_bore()
  use_ip = ""
  if port:
    # !curl ifconfig.me > ip.txt
    # clear_output()
    # ip = getoutput('cat /content/ip.txt')
    # ip = f"-l {ip}"
    !nohup bore local 5432 --to bore.pub -p $port > /content/tunnel_srv.txt 2>&1 &
  else:
    !nohup bore local 5432 --to bore.pub > /content/tunnel_srv.txt 2>&1 &
  time.sleep(3)
  !grep -o 'bore.pub:[^ ]*' /content/tunnel_srv.txt >/content/tunnel_srvr.txt
  time.sleep(3)
  tunnel_srv= getoutput('cat /content/tunnel_srvr.txt')
  print('http://'+tunnel_srv)
  # !rm /content/tunnel_srv.txt
  !rm /content/tunnel_srvr.txt

elif Tunnel_type == "ngrok":
  install_ngrok()
  from pyngrok import ngrok
  # Terminate ngrok port
  ngrok.kill()
  # Set authentication (optional)
  # Get your authentication token via https://dashboard.ngrok.com/auth
  NGROK_AUTH_TOKEN = "2H79LWzHBonanC1xWxoGXW6fvtY_5rD7uShKgACuSLvUwMvJA"
  ngrok.set_auth_token(NGROK_AUTH_TOKEN)
  public_url = ngrok.connect(5432)
  print(public_url)

elif Tunnel_type == "cloudflare":
  install_cloudflare()
  !pkill cloudflared
  time.sleep(4)
  # Tunnel_link = "& cloudflared tunnel --url http://localhost:7860"
  !nohup cloudflared tunnel --url http://localhost:5432 > /content/srv.txt 2>&1 &
  time.sleep(4)
  !grep -o 'https[^[:space:]]*\.trycloudflare.com' /content/srv.txt >/content/srvr.txt
  srv= getoutput('cat /content/srvr.txt')
  print(srv)
  time.sleep(4)
  # !rm /content/srv.txt /content/srvr.txt

In [ ]:
!sed -i 's/#wal_level = replica/wal_level = logical/g' /etc/postgresql/12/main/postgresql.conf

In [ ]:
!sudo service postgresql start

In [ ]:
!ps -ef | grep postgresql

####redis

In [ ]:
!sudo apt install lsb-release

In [ ]:
!curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

!echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

!sudo apt-get update
!sudo apt-get install redis

In [ ]:
!sudo service redis-server start

#exp

In [ ]:
!curl -X GET "localhost:9200"

In [ ]:
!/content/elasticsearch-8.7.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"

In [ ]:
!curl --cacert /content/elasticsearch-8.7.0/config/certs/http_ca.crt -u elastic -H ' Content-Type: application/json' -XGET https://localhost:9200/?pretty=true

In [ ]:
!curl --cacert /content/elasticsearch-8.7.0/config/certs/http_ca.crt -u elastic -H 'Content-Type: application/json' -XPUT https://localhost:9200/playground -d ' { "mappings": { "properties": { "name" : { "type": "keyword" } } } }'

In [ ]:
!curl --cacert /content/elasticsearch-8.7.0/config/certs/http_ca.crt -u elastic -H 'Content-Type: application/json' -XGET https://localhost:9200/playground/_mapping?pretty = true

In [ ]:
!curl --cacert /content/elasticsearch-8.7.0/config/certs/http_ca.crt -u elastic -H 'Content - Type: application/json' -XPOST https://localhost:9200/playground/_doc -d '{"name": "HOGE"}'